In [1]:
%pwd

'c:\\Users\\HP\\Cancer Detection\\research'

In [2]:
import os

os.chdir("../")

%pwd

'c:\\Users\\HP\\Cancer Detection'

### Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



### Configuration Manager

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


### Component

In [8]:
import os
import gdown
import zipfile
from src.cnnClassifier import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_files(self) -> str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok = True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")

        except Exception as e:
            raise e
        

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

### Pipeline Code

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_files()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-07-11 07:48:39,913: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-11 07:48:39,913: INFO: common: created directory at: artifacts]
[2024-07-11 07:48:39,920: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-11 07:48:39,922: INFO: 2579417938: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=dc18b06d-198d-4027-8981-7e5c8edb5304
To: c:\Users\HP\Cancer Detection\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:17<00:00, 2.83MB/s]

[2024-07-11 07:48:59,343: INFO: 2579417938: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into artifacts/data_ingestion/data.zip]
